In [1]:
import xarray as xr

In [2]:
path = "/data/keeling/a/alfonso8/gpm/camp2ex/cloud_probes/zarr"

In [3]:
ds_lear = xr.open_zarr(f'{path}/Page0_Learjet.zarr')

In [12]:
for idx, group in ds_lear.groupby('time.day'):
    print(group.time.min().values)
    print(group.time.max().values)
    print("----------------------------------")

2019-09-07T00:26:55.000000000
2019-09-07T08:26:08.000000000
----------------------------------
2019-09-09T00:44:23.000000000
2019-09-09T04:16:17.000000000
----------------------------------
2019-09-10T00:36:28.000000000
2019-09-10T04:08:02.000000000
----------------------------------
2019-09-13T05:18:55.000000000
2019-09-13T08:29:17.000000000
----------------------------------
2019-09-15T07:25:21.000000000
2019-09-15T10:37:02.000000000
----------------------------------
2019-09-17T00:27:16.000000000
2019-09-17T07:43:03.000000000
----------------------------------
2019-09-20T01:00:56.000000000
2019-09-20T04:37:15.000000000
----------------------------------
2019-09-21T23:36:34.000000000
2019-09-21T23:59:59.000000000
----------------------------------
2019-09-22T00:00:00.000000000
2019-09-22T02:39:13.000000000
----------------------------------
2019-09-24T06:08:45.000000000
2019-09-24T09:04:06.000000000
----------------------------------
2019-09-25T04:17:57.000000000
2019-09-25T07:14:49.

In [13]:
ds_p3b = xr.open_zarr(f'{path}/p3b_merge.zarr')

In [14]:
for idx, group in ds_p3b.groupby('time.day'):
    print(group.time.min().values)
    print(group.time.max().values)
    print("----------------------------------")

2019-10-01T21:40:04.000000000
2019-10-01T23:59:59.000000000
----------------------------------
2019-10-02T00:00:00.000000000
2019-10-02T06:27:59.000000000
----------------------------------
2019-10-03T22:55:09.000000000
2019-10-03T23:59:59.000000000
----------------------------------
2019-09-04T00:01:21.000000000
2019-10-04T03:05:24.000000000
----------------------------------
2019-10-05T01:31:06.000000000
2019-10-05T07:29:24.000000000
----------------------------------
2019-09-06T23:55:24.000000000
2019-09-06T23:59:59.000000000
----------------------------------
2019-09-07T00:00:00.000000000
2019-09-07T08:56:21.000000000
----------------------------------
2019-09-08T23:59:06.000000000
2019-09-08T23:59:59.000000000
----------------------------------
2019-09-09T00:00:00.000000000
2019-09-09T09:01:25.000000000
----------------------------------
2019-09-13T20:41:21.000000000
2019-09-13T23:59:59.000000000
----------------------------------
2019-09-14T00:00:00.000000000
2019-09-14T04:50:24.